# Phase 4 – Generative AI Integration (CIMA Dataset)

## Objective

In this phase, we integrate a Generative AI model to work with the CIMA films dataset.

**Goal:**  
The user writes a description of a film in Arabic, and the AI predicts the most likely **film genre** (تصنيف الفيلم) based on patterns it has learned from data and general knowledge.

We will:

1. Load the CIMA dataset.
2. Extract the list of possible genres from the dataset.
3. Design **two different prompt templates** for genre prediction:
   - Template 1: Short, direct classification.
   - Template 2: Detailed explanation with reasoning.
4. Connect to a Generative AI model (e.g., GPT) via API.
5. Test both templates using:
   - User-like descriptions we create.
   - Real film summaries (ملخص) from the CIMA dataset.
6. Compare the outputs and justify which template is better for the final system.


In [2]:

!pip install -q groq
import os
import pandas as pd


df = pd.read_excel("cleaned_cima_dataset.xlsx")

df.head()







,اسم الفيلم,تاريخ العرض,تصنيف الفيلم,مدة الفيلم (دقيقة),ملخص,تأليف,تمثيل,إنتاج,تصوير,مونتاج,ديكور,ملابس,موسيقى,إخراج,توزيع
0,قلب المرأة,1940,رومانسي,0.062944,تدور أحداث الفيلم حول (خيرية)، الفتاة الثرية ا...,توجو مزراحي (أحمد المشرقي),"سليمان نجيب,أمينة رزق,دولت أبيض,عقيلة راتب,سلو...",الأفلام المتحدة (أنور وجدي وشركاه),عبدالحليم نصر,جلال مصطفى,عباس حلمي,أحمد الكسار,فؤاد الظاهري,توجو مزراحي (أحمد المشرقي),منتخبات بهنا فيلم
1,الورشة,1940,دراما,0.081218,يسافر الأسطى (علي عبدالرحمن) صاحب ورشة الميكان...,"عزيزة أمير,محمود ذو الفقار","عزيزة أمير,محمود ذو الفقار,أنور وجدي,نجمة إبرا...",إيزيس فيلم,"أرام ماراليان,فيرى فاركاش (فرانسوا فاركاش)",جلال مصطفى,محمد كامل,أحمد الكسار,عبدالحميد عبدالرحمن,إستيفان روستي,منتخبات بهنا فيلم
2,تحت السلاح,1940,دراما,0.082185,أمينة فتاة من أسرة عريقة، تقرر العائلة زواجها ...,أبو السعود الإبياري,"أحمد علام,زوزو شكيب,عباس فارس,محمد الديب,زينات...",ألفيزى اورفانيللى,عبدالحليم نصر,جلال مصطفى,عباس حلمي,أحمد الكسار,فؤاد الظاهري,فؤاد الجزايرلي,منتخبات بهنا فيلم
3,حياة الظلام,1940,دراما,0.070051,عائلة متوسطة الحال مكونة من زوج وزوجة وابنهما ...,"محمود كامل حسن,أحمد بدرخان","ميمي شكيب,محسن سرحان,روحية خالد,فردوس محمد,علي...",محمد جمال الدين رفعت,محمد عبدالعظيم,جمال مدكور,"علي عابد,روبرت شارفنبرج",أحمد الكسار,"بيرم التونسي,عبدالحميد عبدالرحمن,محمد الكحلاوي...","أحمد بدرخان,فريد الجندي",منتخبات بهنا فيلم
4,صرخة في الليل,1940,دراما,0.086294,لا تحفظ الزوجة الساقطة شرف زوجها الثري ترتبط ب...,السيد زيادة,"بدر لاما,رجاء عبده,سميحة سميح,بشارة واكيم,منسى...",كوندور فيلم,إبراهيم شيبا,"بدر لاما,ألبير نجيب",محروس زيادة,أحمد الكسار,"فريد غصن,يوسف صالح",إبراهيم لاما,منتخبات بهنا فيلم


In [6]:

from groq import Groq
from google.colab import userdata

api_key = userdata.get('cima')

client = Groq(api_key=api_key)










In [7]:

print(df.columns.tolist())


movies_df = df[[
    "اسم الفيلم",
    "تصنيف الفيلم",
    "ملخص",
    "مدة الفيلم (دقيقة)",
    "تاريخ العرض"
]].dropna(subset=["اسم الفيلم", "تصنيف الفيلم", "ملخص"])

len(movies_df), movies_df.head()


['اسم الفيلم', 'تاريخ العرض', 'تصنيف الفيلم', 'مدة الفيلم (دقيقة)', 'ملخص', 'تأليف', 'تمثيل', 'إنتاج', 'تصوير', 'مونتاج', 'ديكور', 'ملابس', 'موسيقى', 'إخراج', 'توزيع']


(2368,
       اسم الفيلم تصنيف الفيلم  \
 0     قلب المرأة      رومانسي   
 1         الورشة        دراما   
 2     تحت السلاح        دراما   
 3    حياة الظلام        دراما   
 4  صرخة في الليل        دراما   
 
                                                 ملخص  مدة الفيلم (دقيقة)  \
 0  تدور أحداث الفيلم حول (خيرية)، الفتاة الثرية ا...            0.062944   
 1  يسافر الأسطى (علي عبدالرحمن) صاحب ورشة الميكان...            0.081218   
 2  أمينة فتاة من أسرة عريقة، تقرر العائلة زواجها ...            0.082185   
 3  عائلة متوسطة الحال مكونة من زوج وزوجة وابنهما ...            0.070051   
 4  لا تحفظ الزوجة الساقطة شرف زوجها الثري ترتبط ب...            0.086294   
 
    تاريخ العرض  
 0         1940  
 1         1940  
 2         1940  
 3         1940  
 4         1940  )

In [8]:

def build_candidate_movies(
    df,
    preferred_genre=None,
    max_duration=None,
    min_year=None,
    max_year=None,
    n_candidates=8
):
    data = df.copy()

    if preferred_genre:
        data = data[data["تصنيف الفيلم"].str.contains(preferred_genre, na=False)]


    if max_duration:
        data = data[data["مدة الفيلم (دقيقة)"] <= max_duration]


    if (min_year is not None) or (max_year is not None):
        years = (
            data["تاريخ العرض"]
            .astype(str)
            .str.extract(r"(\d{4})")[0]
            .astype(float)
        )
        data = data.copy()
        data["سنة العرض"] = years

        if min_year is not None:
            data = data[data["سنة العرض"] >= min_year]
        if max_year is not None:
            data = data[data["سنة العرض"] <= max_year]


    if len(data) < n_candidates:
        data = df.copy()


    candidates = data.sample(n=min(n_candidates, len(data)), random_state=42)

    return candidates[["اسم الفيلم", "تصنيف الفيلم", "ملخص", "مدة الفيلم (دقيقة)"]]


In [9]:


GENRES_LIST = ", ".join(sorted(movies_df["تصنيف الفيلم"].dropna().unique().tolist()))


TEMPLATE_A = """
أنت نظام توصية أفلام عربي.

سيتم تزويدك بمعلومات عن حالة المستخدم، وقائمة من الأفلام المصرية (اسم، تصنيف، مدة، ملخص).
مهمتك أن تختار فيلماً واحداً فقط هو الأنسب للمستخدم، ثم تشرح له سبب الاختيار.

المستخدم:
{user_profile}

الأفلام المرشحة:
{movies_block}

التعليمات:
- اختر أفضل فيلم واحد فقط للمستخدم.
- لا تختر فيلماً غير موجود في القائمة.
- اشرح السبب بشكل مختصر وواضح.
- اذكر إذا كان الفيلم مناسب للمشاهدة مع العائلة أو لا (إذا أمكن استنتاج ذلك من الملخص).

صيغة الإجابة المطلوبة:

الفيلم المقترح: <اسم الفيلم>
التصنيف: <تصنيف الفيلم>
سبب الترشيح: <شرح مختصر>
مناسب للعائلة؟: <نعم/لا/غير واضح>
"""


TEMPLATE_B = """
أنت مستشار سينمائي خبير في الأفلام المصرية.

ستصلك حالة المستخدم وقائمة بالأفلام المرشحة (اسم، تصنيف، مدة، ملخص).
مهمتك:
1. ترتيب أفضل ثلاثة أفلام من القائمة من الأنسب إلى الأقل مناسبة.
2. توضيح سبب اختيار كل فيلم، مع ربطه بحالة المستخدم (المزاج، الوقت المتاح، التصنيف المفضل).
3. إعطاء نصيحة عامة للمستخدم عن نوع الأفلام الذي يناسبه مستقبلاً.

المستخدم:
{user_profile}

الأفلام المرشحة:
{movies_block}

التعليمات:
- التزم فقط بالأفلام الموجودة في القائمة.
- استخدم اللغة العربية الفصحى المبسطة.
- اربط التوصية مباشرة بحالة المستخدم (المزاج، الوقت، نوع الفيلم).

صيغة الإجابة المقترحة:

1) <اسم الفيلم 1> - السبب:
2) <اسم الفيلم 2> - السبب:
3) <اسم الفيلم 3> - السبب:

نصيحة عامة:
<نصيحة قصيرة عن نوع الأفلام المناسب للمستخدم>
"""


In [10]:
def call_llm(template, user_profile_text, candidate_movies_df, model="llama-3.1-8b-instant"):
    movies_lines = []
    for _, row in candidate_movies_df.iterrows():
        line = (
            f"- اسم الفيلم: {row['اسم الفيلم']}\n"
            f"  التصنيف: {row['تصنيف الفيلم']}\n"
            f"  المدة: {row['مدة الفيلم (دقيقة)']} دقيقة\n"
            f"  الملخص: {row['ملخص']}\n"
        )
        movies_lines.append(line)

    movies_block = "\n".join(movies_lines)

    prompt = template.format(
        user_profile=user_profile_text,
        movies_block=movies_block
    )

    try:
        response = client.chat.completions.create(
            model=model,
            messages=[{"role": "user", "content": prompt}],
            temperature=0.4
        )
        return response.choices[0].message.content.strip()
    except Exception as e:
        return f"[Error: {e}]"


In [11]:

user_profile_1 = """
مزاجي اليوم خفيف وأريد مشاهدة فيلم كوميدي أو رومانسي خفيف أقدّره مع العائلة.
ما عندي وقت طويل، أفضّل أن تكون مدة الفيلم أقل من ساعتين.
"""

candidates_1 = build_candidate_movies(
    movies_df,
    preferred_genre="كوميدي",
    max_duration=120,
    min_year=None,
    max_year=None,
    n_candidates=8
)

print("عدد الأفلام المرشحة:", len(candidates_1))
candidates_1[["اسم الفيلم", "تصنيف الفيلم", "مدة الفيلم (دقيقة)"]]


عدد الأفلام المرشحة: 8


,اسم الفيلم,تصنيف الفيلم,مدة الفيلم (دقيقة)
509,شمشون ولبلب (عنتر ولبلب),كوميدي,0.093401
2054,صح النوم,كوميدي,0.096447
1471,حلوة وشقية,كوميدي,0.078173
992,زوج بالإيجار,كوميدي,0.090355
795,ابن حميدو,كوميدي,0.086294
1635,عريس بنت الوزير,كوميدي,0.086294
1468,7 أيام في الجنة,كوميدي,0.096447
2327,خلي بالك من جيرانك,كوميدي,0.110660


In [12]:

advice_A_1 = call_llm(TEMPLATE_A, user_profile_1, candidates_1)
advice_B_1 = call_llm(TEMPLATE_B, user_profile_1, candidates_1)

print("=========== TEMPLATE A (نصيحة مباشرة) ===========")
print(advice_A_1)

print("\n\n=========== TEMPLATE B (ترتيب + شرح مفصل) ===========")
print(advice_B_1)


=========== TEMPLATE A (نصيحة مباشرة) ===========
الفيلم المقترح: شمشون ولبلب
التصنيف: كوميدي
سبب الترشيح: هذا الفيلم مناسب للمستخدم لأنها يبحث عن فيلم كوميدي خفيف يمكنه مشاهدة مع العائلة، ومدة الفيلم أقل من ساعتين، وهو أيضًا فيلم مصرى يعتبر من الأفضل في هذا النوع.
مناسب للعائلة؟: نعم


=========== TEMPLATE B (ترتيب + شرح مفصل) ===========
1) <اسم الفيلم 1>: شمشون ولبلب - السبب: هذا الفيلم مناسب للمستخدم بسبب كوميديته الخفيفة والرومانسية الموجودة فيه. كما أنه يحتوي على شخصيات محببة ومفارقات كوميدية ساخرة. يعتبر هذا الفيلم مناسباً للمزاج الخفيف للمستخدم ويمكن مشاهدته مع العائلة.

2) <اسم الفيلم 2>: حلوة وشقية - السبب: هذا الفيلم مناسب للمستخدم بسبب كوميديته الخفيفة والرومانسية الموجودة فيه. كما أنه يحتوي على شخصيات محببة ومفارقات كوميدية ساخرة. يعتبر هذا الفيلم مناسباً للمزاج الخفيف للمستخدم ويمكن مشاهدته مع العائلة.

3) <اسم الفيلم 3>: خلي بالك من جيرانك - السبب: هذا الفيلم مناسب للمستخدم بسبب كوميديته الخفيفة والرومانسية الموجودة فيه. كما أنه يحتوي على شخصيات محببة ومفارقات كوميدية سا

In [13]:

scenarios = [
    {
        "name": "مزاج خفيف مع العائلة",
        "profile": """
        أبحث عن فيلم كوميدي أو عائلي أتابعه مع العائلة في عطلة نهاية الأسبوع.
        أريد جو خفيف بدون عنف أو محتوى غير مناسب للأطفال.
        """,
        "genre": "كوميدي",
        "max_duration": 130
    },
    {
        "name": "دراما عاطفية",
        "profile": """
        أريد فيلم درامي عاطفي عن العلاقات أو الصراعات الأسرية،
        ما يهمني إذا كان الفيلم قديم أو جديد، لكن أفضّل أن تكون القصة عميقة.
        """,
        "genre": "دراما",
        "max_duration": None
    }
]

results = []

for sc in scenarios:
    cand = build_candidate_movies(
        movies_df,
        preferred_genre=sc["genre"],
        max_duration=sc["max_duration"],
        n_candidates=8
    )

    outA = call_llm(TEMPLATE_A, sc["profile"], cand)
    outB = call_llm(TEMPLATE_B, sc["profile"], cand)

    results.append({
        "Scenario": sc["name"],
        "Template": "A",
        "Advice": outA
    })
    results.append({
        "Scenario": sc["name"],
        "Template": "B",
        "Advice": outB
    })

results_df = pd.DataFrame(results)
results_df


,Scenario,Template,Advice
0,مزاج خفيف مع العائلة,A,الفيلم المقترح: شمشون ولبلب\nالتصنيف: كوميدي\n...
1,مزاج خفيف مع العائلة,B,أنا مستشار سينمائي خبير في الأفلام المصرية، وس...
2,دراما عاطفية,A,الفيلم المقترح: الدموع الساخنة\nالتصنيف: دراما...
3,دراما عاطفية,B,1) الدموع الساخنة - السبب: يعتبر الفيلم الدرام...


In this phase, we integrated Generative AI into our Arabic Movie Recommendation System to transform it from a simple filtering model into a real advice/action-suggestion system.

We first cleaned and prepared the Egyptian Arabic movies dataset, then built a classical filtering step to narrow down candidate movies based on user preferences such as genre, duration, and release year.

After that, we designed two different prompt templates to test how prompt design affects the quality of recommendations:

Template A: provides one recommended movie with a short explanation.

Template B: provides the top three movies, ranked with detailed reasoning and a general viewing tip.

The user interacts with the system through a natural-language input (e.g., “I want a light comedy suitable for family and not too long”). This input is combined with the candidate movies and sent to the LLM (Llama 3.1 via Groq API), which generates personalized, natural-sounding movie advice in Arabic.

Results:
Template A was fast and concise, but Template B produced richer, more helpful recommendations with deeper reasoning. Overall, Template B delivered the best user experience and aligned more strongly with the “advice/action-suggestion” requirement.

In conclusion, integrating Generative AI significantly improved the recommendation system, making it more human-like, contextual, and personalized.